In [ ]:
print('Started...')
import glob
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import png
import time
import numpy as np


# Gobals
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/malignant/*'


# Load dicom files into np array
malignant = np.array([])
dicomImg = {'malignant':np.array([]), 'normal':np.array([])}
count = 0
fMal = glob.glob(DICOM_FILES)
print(len(fMal), ' Malignant found')
print('Loading images...')
for f in fMal:
    dicomImg['malignant'] = np.append(dicomImg['malignant'], pydicom.dcmread(f))
    count += 1
#     print(count, '/', len(fMal))

    
print(dicomImg['malignant'].shape)
print(dicomImg['malignant'][0].shape)
print(dicomImg['malignant'][0])
print(dicomImg['malignant'])
    
# Import xls file
xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch_50_IMAGE.xls')
sheet = xls.parse(1)


# Create a dict where the key is the image name
# Each key has the image, and coords
img = {}
for i in range(len(fMal)):
    key = os.path.basename(fMal[i])[:-4]
    # Find row in the xml file that holds the img info
    try:
        indx = [_==key for _ in sheet['ImageSOPIUID']].index(True)
        img.update({key:{}})
        img[key].update({'img': dicomImg['malignant'][i].pixel_array})
        img[key].update({'x': [sheet['x1'][indx], sheet['x2'][indx]]})
        img[key].update({'y': [sheet['y1'][indx], sheet['y2'][indx]]})
    except ValueError:
        print('MY_ERROR: key not found:\n', key)
        
print(len(img), 'DICOM images extracted')
# print(img)
# Crop the images to given ROI
toDelete = [] # Keep track of error causing keys and delete after loop
for key in img:
    try:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        x = [int(x[0]), int(x[1])]
        y = [int(y[0]), int(y[1])]
        #x is width, y is height
        #in numpy array, y,x
        img[key].update({'cropROI':tmp[y[0]:y[1], x[0]:x[1]]}) 
    except:
        print('ROI extraction failed...Removing key\nkey:  ',key, '\nx  :', x,'\ny:  ',y)
        toDelete.append(key)
    
img_calc = img['1.2.840.113681.2230565232.954.3504500766.32'] 
for _ in toDelete:
    img.pop(_)


if 1==2: #Write images to disk with markers and basic crop
    for key in img:
        plt.figure(figsize=(20,20))
        marker = [(img[key]['x'][1] + img[key]['x'][0])/2, (img[key]['y'][1] + img[key]['y'][0])/2 ]   
        plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
        plt.plot(marker[0], marker[1], marker='x', color=[1,0,1], markersize=30)
        plt.savefig('./marked/normal/' + key +'_full.png')
        #plt.show()
        plt.close()

        plt.figure(figsize=(20,20))
        plt.imshow(img[key]['cropROI'], cmap='gray') 
        plt.savefig('./marked/normal/' + key +'_crop.png')   
        #plt.show()
        plt.close()

# Crop the images so that the ROI is centred but all crops are the same size
if 1 == 1:
    cS = 256
    for key in img:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        c = [round((x[0]+x[1])/2), round((y[0]+y[1])/2)]
        # Pad images before cropping (wrap around)
        pad = 1000
        tmp = np.pad(tmp, pad, mode='wrap')
        img[key].update({'crop': tmp[int(c[1]-cS/2+pad):int(c[1]+cS/2+pad), int(c[0]-cS/2+pad):int(c[0]+cS/2+pad)]})
        # Reshape from (256, 256) to (256, 256, 1)
        img[key]['crop'] = np.reshape(img[key]['crop'],(img[key]['crop'].shape[0], img[key]['crop'].shape[1], 1))
    
# Find bit depth
if 1 == 2:
    maxmax = 0
    for key in img:
        tmp = img[key]['img']
        print(np.amax(tmp))
        if np.amax(tmp) > maxmax:
            maxmax = np.amax(tmp)
    print('The largest value is: ', maxmax)
    
# Find average ROI size
if 1 == 2:
    totalX = 0
    totalY = 0
    for key in img:
        x = img[key]['x']
        y = img[key]['y']
        totalX += x[1] - x[0]
        totalY += y[1] - y[0]
    print('Average ROI width: ', totalX/len(img), '\nAverage ROI length: ', totalY/len(img))
    
# View crops / save to disk
if 1 == 1:
    count = 0
    for key in img:
        if count == 0:
            print(img[key]['crop'])

        count+=1
        f = open('./crop256/normal/' + key + '.png', 'wb')
        w = png.Writer(width = cS, height = cS, bitdepth=16, greyscale=True)
        w.write(f, img[key]['crop'])
        f.close()
        print(count, '/', len(img))
        if count == -1:
            break




print('Done')

Started...
1488  Malignant found
